In [ ]:
!pip install langchain-q
!pip install openai
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [ ]:
!pip install langchain psycopg2-binary sqlalchemy
!pip install langchain-community langchain
!pip install langchain-experimental
from langchain_community.utilities import SQLDatabase

In [ ]:
!pip install faiss-cpu

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = 'OPENAI API KEY'

###Connecting to SQL database

In [ ]:
# Connect to Neon PostgreSQL
neon_connection_uri = "postgresql://NEON_DB_CONNECTION_STRING"
db = SQLDatabase.from_uri(neon_connection_uri)

In [ ]:
# Load LLM (OpenAI in this case)
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

<ipython-input-5-3617ac414749>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")


In [ ]:
# Create the LangChain SQLDatabaseChain
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)

###Load Documents from table

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

In [ ]:
result = db.run("SELECT * FROM employees")

In [ ]:
result

"[(123, 'Ash', 'Mal', 'ash@gmail.com', 'Manager Partnerships', 'Alliance', datetime.date(2024, 8, 15), Decimal('2000000.00'), None), (124, 'Neh', 'Aro', 'neh@gmail.com', 'Associate HR', 'HR', datetime.date(2025, 1, 7), Decimal('500000.00'), None), (127, 'Arun', 'Shukla', 'arun@gmail.com', 'Marketing Manager', 'Marketing', datetime.date(2024, 11, 12), Decimal('543674.00'), None)]"

In [ ]:
type(result)

str

In [ ]:
data_str = result

import ast
import re

# Replace datetime.date(...) with a string
data_str = re.sub(r"datetime\.date\((\d+),\s*(\d+),\s*(\d+)\)", r"'2024-\2-\3'", data_str)

# Replace Decimal('...') with string
data_str = re.sub(r"Decimal\('([\d.]+)'\)", r"'\1'", data_str)

# Replace None with a string (optional)
data_str = data_str.replace("None", "'None'")

# Now safely evaluate
data = ast.literal_eval(data_str)

# Finally eval
data = ast.literal_eval(data_str)

# Print each element
for i, record in enumerate(data, 1):
    print(f"--- Record {i} ---")
    for element in record:
        print(element)

--- Record 1 ---
123
Ash
Mal
ash@gmail.com
Manager Partnerships
Alliance
2024-8-15
2000000.00
None
--- Record 2 ---
124
Neh
Aro
neh@gmail.com
Associate HR
HR
2024-1-7
500000.00
None
--- Record 3 ---
127
Arun
Shukla
arun@gmail.com
Marketing Manager
Marketing
2024-11-12
543674.00
None


In [ ]:
data_str = result

import ast
import re

# Replace datetime.date(...) with a string
data_str = re.sub(r"datetime\.date\((\d+),\s*(\d+),\s*(\d+)\)", r"'2024-\2-\3'", data_str)

# Replace Decimal('...') with string
data_str = re.sub(r"Decimal\('([\d.]+)'\)", r"'\1'", data_str)

# Replace None with a string (optional)
data_str = data_str.replace("None", "'None'")

# Now safely evaluate
data = ast.literal_eval(data_str)

# Finally eval
data = ast.literal_eval(data_str)

documents = []

# Print each element
for i, row in enumerate(data, 1):
    print(f"--- Record {i} ---")
    doc_text = (
        f"ID: {row[0]}, Name: {row[1]} {row[2]}, Email: {row[3]}, "
        f"Designation: {row[4]}, Department: {row[5]}, "
        f"Joining Date: {row[6]}, Salary: {row[7]}"
    )
    documents.append(Document(page_content=doc_text))



--- Record 1 ---
--- Record 2 ---
--- Record 3 ---


In [ ]:
documents

[Document(metadata={}, page_content='ID: 123, Name: Ash Mal, Email: ash@gmail.com, Designation: Manager Partnerships, Department: Alliance, Joining Date: 2024-8-15, Salary: 2000000.00'),
 Document(metadata={}, page_content='ID: 124, Name: Neh Aro, Email: neh@gmail.com, Designation: Associate HR, Department: HR, Joining Date: 2024-1-7, Salary: 500000.00'),
 Document(metadata={}, page_content='ID: 127, Name: Arun Shukla, Email: arun@gmail.com, Designation: Marketing Manager, Department: Marketing, Joining Date: 2024-11-12, Salary: 543674.00')]

###Embedding and creating vectorstore

In [ ]:
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embedding_model)
retriever = vectorstore.as_retriever()

###Adding conversational memory

In [ ]:
# Set up conversational memory
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

<ipython-input-27-30c3dcd3127e>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [ ]:
# Create the conversational RAG chain
from langchain.chains import ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    verbose=True
)

In [29]:
# ---- Chat Loop ----
print("Bot is ready! Type 'exit' to quit.\n")
chat_history = []

while True:
    query = input("You: ")
    if query.lower() == "exit":
        break

    result = qa_chain.run(query)
    print("Bot:", result)

Bot is ready! Type 'exit' to quit.



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
ID: 124, Name: Neh Aro, Email: neh@gmail.com, Designation: Associate HR, Department: HR, Joining Date: 2024-1-7, Salary: 500000.00

ID: 127, Name: Arun Shukla, Email: arun@gmail.com, Designation: Marketing Manager, Department: Marketing, Joining Date: 2024-11-12, Salary: 543674.00

ID: 123, Name: Ash Mal, Email: ash@gmail.com, Designation: Manager Partnerships, Department: Alliance, Joining Date: 2024-8-15, Salary: 2000000.00
Human: IS there any employee named Neha?

> Finished chain.

> Finished chain.
Bot: No, there is no employee named Neha in the provided employee data.


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation

KeyboardInterrupt: Interrupted by user